In [7]:
using BenchmarkTools
using LoopVectorization

n_clusters = 32
n_examples = 1_000_000
n_features = 128

T = rand(n_clusters, n_features);                   # ADC table
PQ = rand(1:n_clusters, n_features, n_examples)    # PQcodes
PQ_trans =  Matrix(PQ');
y = rand(n_features);


function lsh_transposed(PQ_trans, T)
    
    n_examples, n_features = size(PQ_trans)
    d = zeros(eltype(T), n_examples)
    @inbounds @fastmath for j in 1:n_features
        for n in 1:n_examples
            d[n] += T[ PQ_trans[n,j], j ]    
        end
    end
    return d
end


function lsh(PQ, T)
    
    n_features, n_examples = size(PQ)
    d = zeros(eltype(T), n_examples)
    
    @inbounds for n in 1:n_examples
        res = zero(eltype(T))
        @fastmath for j in 1:n_features
            res += T[PQ[j,n],j]    
        end
        d[n] = res
    end
    return d
end

res_lsh_transposed = lsh_transposed(PQ_trans, T);
res_lsh = lsh(PQ, T);

@assert isapprox(res_lsh_transposed, res_lsh)
@benchmark lsh($PQ, $T)


BenchmarkTools.Trial: 117 samples with 1 evaluation.
 Range (min … max):  42.106 ms …  44.896 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     42.641 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   42.731 ms ± 285.077 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                   ▄ ██▁                                        
  ▃▁▁▁▁▁▁▁▁▁▁▁▁▃▁▃▄█▇████▅▆▃▅▅▅▇▅█▆█▄▃▃▅▁▃▃▄▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▃ ▃
  42.1 ms         Histogram: frequency by time         43.5 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [8]:
@benchmark lsh_transposed($PQ_trans, $T)

BenchmarkTools.Trial: 64 samples with 1 evaluation.
 Range (min … max):  77.849 ms …  81.274 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     79.327 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   79.302 ms ± 465.395 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                   ▃ ▁ ██  ▁                    
  ▄▁▁▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▄▁▄▄▄▆▁▁▄▇█▆█▇██▆▄█▇▆▄▄▁▄▄▁▁▄▁▁▁▁▁▁▄▄ ▁
  77.8 ms         Histogram: frequency by time         80.2 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

## Other tests

In [140]:
using StaticArrays

PQ = [SVector{n_features,UInt8}(rand(1:n_clusters, n_features)) for i in 1:n_examples]

function lsh_static(PQ, T)
     n_features, n_examples = length(PQ[1]), length(PQ)
     d = Array{eltype(T)}(undef, n_examples)
     @inbounds for n in 1:n_examples
         res = zero(eltype(T))
         p = PQ[n]
         @simd for j in 1:n_features
             res += T[p[j],j]
         end
         d[n] = res
     end
     return d
 end

lsh_static (generic function with 1 method)

In [141]:
@benchmark lsh_static($PQ, $T)

BenchmarkTools.Trial: 113 samples with 1 evaluation.
 Range (min … max):  43.712 ms …  46.491 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     44.183 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   44.406 ms ± 520.554 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▃█                                                     
  ▄▁▁▁▁▇▅██▇█▄▃▄▃▃▄▃▃▃▆▆▁▁▅▃▃▄▃▁▁▃▁▃▁▃▁▄▁▃▁▁▁▁▁▁▁▁▁▃▁▃▁▁▁▁▁▁▁▃ ▃
  43.7 ms         Histogram: frequency by time         46.3 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

Maybe using a dict...

In [128]:
T_dict = Dict()

for n in 1:n_clusters
    for j in 1:n_features
        T_dict[(n,j)] = T[n,j]
    end
end

In [131]:

function lsh_dict(PQ, T_dict::Dict)
    
    n_features, n_examples = size(PQ)
    d = zeros(eltype(T_dict[1,1]), n_examples)
    
    @inbounds @fastmath for n in 1:n_examples
        res = zero(eltype(T))
        for j in 1:n_features
            res += T_dict[PQ[j,n],j]    
        end
        d[n] = res
    end
    return d
end

lsh_dict (generic function with 1 method)

In [ ]:
res_lsh_dict = lsh_dict(PQ, T_dict)